In [21]:
import xarray as xr
import pathlib
import sys
import datetime
from data_config import (
    get_scratch_dir,
    get_dask_log_dir,
    get_dask_local_dir,
)

from process_files import (
    memory,
    get_case_metadata,
)
import numpy as np
import zarr

import pandas as pd
import tqdm
import dask.array as da

In [22]:
scratch = get_scratch_dir()
dask_log_directory = get_dask_log_dir()
dask_local_directory = get_dask_local_dir()

In [23]:
parent_dir = pathlib.Path.cwd().parent
sys.path.append(str(parent_dir))

In [24]:
import atlas

In [25]:
@memory.cache
def get_done_cases_df(today=datetime.datetime.today().date()):
    calc = atlas.global_irf_map(cdr_forcing="DOR", vintage="001")

    data = calc.df_case_status
    done = data.loc[data.archive]

    done_cases = done.index.to_list()
    done_cases.remove("smyle.cdr-atlas-v0.control.001")
    done_cases = sorted(done_cases)

    df = calc.df.loc[done_cases]
    return df, done_cases

In [26]:
%%time

df, done_cases = get_done_cases_df()
df

CPU times: user 5.41 ms, sys: 3.13 ms, total: 8.54 ms
Wall time: 11 ms


,blueprint,polygon,polygon_master,basin,start_date,cdr_forcing,cdr_forcing_file,simulation_key,refdate,stop_n,wallclock,curtail_output
case,,,,,,,,,,,,
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-01-01_00000.001,smyle,0.0,0.0,North_Atlantic_basin,1999-01,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-01-01_00000,0347-01-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-04-01_00001.001,smyle,0.0,0.0,North_Atlantic_basin,1999-04,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-04-01_00001,0347-04-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-07-01_00002.001,smyle,0.0,0.0,North_Atlantic_basin,1999-07,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-07-01_00002,0347-07-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-10-01_00003.001,smyle,0.0,0.0,North_Atlantic_basin,1999-10,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-10-01_00003,0347-10-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_001_1999-01-01_00004.001,smyle,1.0,1.0,North_Atlantic_basin,1999-01,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_001_1999-01-01_00004,0347-01-01,15,10:00:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...
smyle.cdr-atlas-v0.glb-dor_South_037_1999-01-01_01548.001,smyle,37.0,387.0,South,1999-01,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_South_037_1999-01-01_01548,0347-01-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_South_037_1999-04-01_01549.001,smyle,37.0,387.0,South,1999-04,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_South_037_1999-04-01_01549,0347-04-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_South_037_1999-10-01_01551.001,smyle,37.0,387.0,South,1999-10,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_South_037_1999-10-01_01551,0347-10-01,15,10:00:00,True


In [27]:
base_directory = pathlib.Path(
    "/global/cfs/projectdirs/m4746/Projects/Ocean-CDR-Atlas-v0/data/analysis"
)
base_directory

PosixPath('/global/cfs/projectdirs/m4746/Projects/Ocean-CDR-Atlas-v0/data/analysis')

In [32]:
def add_additional_coords(ds: xr.Dataset, case: str, case_metadata: pd.Series):
    polygon_master = int(case_metadata.polygon_master)
    if polygon_master < 0 or polygon_master > 689:
        raise ValueError(
            f"Polygon id must be in range [0, 690). Found polygon_id={polygon_master}"
        )

    # add as an integer coordinate
    polygon_id_coord = xr.DataArray(
        name="polygon_id",
        dims="polygon_id",
        data=[polygon_master],
        attrs={"long_name": "polygon ID"},
    ).astype("int32")

    # injenction date
    injection_date_coord = xr.DataArray(
        data=[int(case_metadata.start_date.split("-")[-1])],
        dims=["injection_date"],
        attrs={"long_name": "injection date", "units": "month of 1999"},
    ).astype("int32")

    renamed = ds.drop_vars("time").rename_dims(time="elapsed_time")

    return renamed.assign_coords(
        polygon_id=polygon_id_coord,
        injection_date=injection_date_coord,
    )


def expand_ensemble_dims(ds: xr.Dataset) -> xr.Dataset:
    """Add new dimensions across the ensemble"""

    copied = ds.copy()

    # all data variables should be ensemble variables
    for name in list(ds.data_vars):
        copied[name] = copied[name].expand_dims(["polygon_id", "injection_date"])

    # absolute time is a function of injection_date because of the different starting times
    # copied["time"] = copied["time"].expand_dims(["injection_date"])
    # copied["time_bound"] = copied["time_bound"].expand_dims(["injection_date"])

    return copied


def compute_dor_efficiency(ds: xr.Dataset) -> xr.Dataset:
    ds["DOR_efficiency"] = -ds.DIC_ADD_TOTAL / ds.DIC_FLUX
    return ds


def set_compression_encoding(ds: xr.Dataset) -> xr.Dataset:
    compressor = zarr.Zlib(level=1)

    for name, var in ds.variables.items():
        # avoid using NaN as a fill value, and avoid overflow errors in encoding
        if np.issubdtype(var.dtype, np.integer):
            ds[name].encoding = {"compressor": compressor, "_FillValue": 2_147_483_647}
        elif var.dtype == np.dtype("float32"):
            ds[name].encoding = {
                "compressor": compressor,
                "_FillValue": 9.969209968386869e36,
            }
        else:
            ds[name].encoding = {"compressor": compressor}

    return ds


def set_elapsed_time(ds: xr.Dataset):
    elapsed_time_integer_months = xr.DataArray(
        np.arange(180), dims=["elapsed_time"], attrs={"units": "months"}
    )
    ds["elapsed_time"] = elapsed_time_integer_months.astype("int32")
    return ds


def process_case(case: str, df: pd.DataFrame) -> xr.Dataset:
    case_metadata = get_case_metadata(case, df=df)
    path = base_directory / f"{case}.analysis.zarr"
    if not path.exists():
        return None
    ds = (
        xr.open_zarr(path, decode_timedelta=True)
        .pipe(add_additional_coords, case, case_metadata)
        .pipe(expand_ensemble_dims)
        .pipe(compute_dor_efficiency)
        .pipe(set_elapsed_time)
    )
    return ds  # [["DOR_efficiency", "elapsed_time"]].drop_vars(["time_delta"])

In [33]:
def create_empty_target_store():
    store1b_chunks_encoding_per_variable = {
        "DOR_efficiency": {
            "chunks": {"polygon_id": 1, "injection_date": 1, "elapsed_time": 180}
        },  # polygon_id: 1 injection_date: 1 elapsed_time: 180
        "polygon_id": {"chunks": {"polygon_id": 690}},  # polygon_id: 1
        "injection_date": {"chunks": {"injection_date": 1}},  # injection_date: 1
        "elapsed_time": {"chunks": {"elapsed_time": 180}},  # elapsed_time: 180
    }
    sizes_all_dims = {
        "elapsed_time": 180,
        "polygon_id": 690,
        "injection_date": 4,
    }

    placeholder = xr.Dataset()
    placeholder["elapsed_time"] = xr.DataArray(
        np.arange(180).astype("int32"), dims=["elapsed_time"], attrs={"units": "months"}
    )
    placeholder["polygon_id"] = xr.DataArray(
        np.arange(690).astype("int32"),
        dims=["polygon_id"],
        attrs={"long_name": "Polygon ID"},
    )
    placeholder["injection_date"] = xr.DataArray(
        np.array([1, 4, 7, 10], dtype="int32"),
        dims=["injection_date"],
        attrs={"long_name": "injection date", "units": "month of 1999"},
    )

    var_chunks = store1b_chunks_encoding_per_variable["DOR_efficiency"]["chunks"]
    var_dims = list(var_chunks.keys())
    var_sizes = {d: s for d, s in sizes_all_dims.items() if d in var_dims}
    var_shape = tuple(var_sizes.values())
    ordered_var_dims = list(var_sizes.keys())

    placeholder["DOR_efficiency"] = xr.DataArray(
        da.empty(
            shape=var_shape,
            chunks=var_chunks,
            dtype="float64",
        ),
        dims=ordered_var_dims,
    )
    placeholder = (
        placeholder.pipe(set_compression_encoding)
        .chunk(polygon_id=1, injection_date=1, elapsed_time=-1)
        .transpose("elapsed_time", "polygon_id", "injection_date")
    )

    return placeholder

In [34]:
store_path = "s3://carbonplan-share/dor-efficiency/store2.zarr"

In [35]:
%%time

cases_without_data = []
for case in tqdm.tqdm(done_cases[:2]):
    single_ds = process_case(case=case, df=df)
    if single_ds:
        pass
        # single_ds.to_zarr(store_path, region="auto")
    else:
        cases_without_data.append(case)

# ds = xr.combine_by_coords(dsets, combine_attrs='drop_conflicts').chunk(polygon_id=-1, injection_date=1, elapsed_time=-1).transpose('elapsed_time', 'polygon_id', 'injection_date')
# ds

100%|██████████| 2/2 [00:00<00:00, 41.06it/s]

CPU times: user 51.1 ms, sys: 0 ns, total: 51.1 ms
Wall time: 50.2 ms


In [36]:
single_ds

<xarray.Dataset> Size: 625MB
Dimensions:         (polygon_id: 1, injection_date: 1, elapsed_time: 180,
                     nlat: 384, nlon: 320)
Coordinates:
    AREA_M2         (nlat, nlon) float64 983kB dask.array<chunksize=(192, 160), meta=np.ndarray>
    KMT             (nlat, nlon) float64 983kB dask.array<chunksize=(192, 160), meta=np.ndarray>
    TLAT            (nlat, nlon) float64 983kB dask.array<chunksize=(192, 160), meta=np.ndarray>
    TLONG           (nlat, nlon) float64 983kB dask.array<chunksize=(192, 160), meta=np.ndarray>
    time_delta      (elapsed_time) timedelta64[ns] 1kB dask.array<chunksize=(180,), meta=np.ndarray>
  * polygon_id      (polygon_id) int32 4B 0
  * injection_date  (injection_date) int32 4B 4
  * elapsed_time    (elapsed_time) int32 720B 0 1 2 3 4 ... 175 176 177 178 179
Dimensions without coordinates: nlat, nlon
Data variables: (12/13)
    ALK_ADD_SURF    (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    ALK_FLUX        (polygon_id, injection_date, elapsed_time) float64 1kB dask.array<chunksize=(1, 1, 180), meta=np.ndarray>
    ALK_FLUX_TOTAL  (polygon_id, injection_date, nlat, nlon) float64 983kB dask.array<chunksize=(1, 1, 192, 160), meta=np.ndarray>
    ALK_SURF        (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    DIC_ADD_SURF    (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    DIC_ADD_TOTAL   (polygon_id, injection_date, elapsed_time) float64 1kB dask.array<chunksize=(1, 1, 180), meta=np.ndarray>
    ...              ...
    DIC_FLUX_TOTAL  (polygon_id, injection_date, nlat, nlon) float64 983kB dask.array<chunksize=(1, 1, 192, 160), meta=np.ndarray>
    DIC_SURF        (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    FG_CO2_ADD      (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    PH_ADD_SURF     (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    pCO2_ADD_SURF   (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    DOR_efficiency  (polygon_id, injection_date, elapsed_time) float64 1kB dask.array<chunksize=(1, 1, 180), meta=np.ndarray>
Attributes:
    case:     smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-04-01_...

In [37]:
def calc_differences(ds: xr.Dataset) -> xr.Dataset:
    # difference = experimental - counterfactual
    ds["delta_ALK"] = ds["ALK"] - ds["ALK_ALT_CO2"]
    ds["delta_DIC"] = ds["DIC"] - ds["DIC_ALT_CO2"]
    ds["delta_PH"] = ds["PH"] - ds["PH_ALT_CO2"]
    ds["delta_FG"] = ds["FG_CO2"] - ds["FG_ALT_CO2"]
    ds["delta_pCO2SURF"] = ds["pCO2SURF"] - ds["pCO2SURF_ALT_CO2"]

    return ds

In [40]:
dt = xr.open_datatree(
    "https://carbonplan-oae-efficiency.s3.us-west-2.amazonaws.com/store2.zarr/",
    engine="zarr",
    chunks={},
)
dt

<xarray.DataTree>
Group: /
│   Attributes:
│       multiscales:  [{'datasets': [{'crs': 'EPSG:3857', 'level': 0, 'path': '0'...
│       title:        multiscale data pyramid
│       version:      0.1.0.post14+gaf6f2fb.d20240522
├── Group: /0
│       Dimensions:         (polygon_id: 690, band: 2, injection_date: 4, month: 12,
│                            year: 15, y: 128, x: 128, y_b: 129, x_b: 129,
│                            z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
│       Coordinates: (12/13)
│         * band            (band) <U12 96B 'delta' 'experimental'
│           elapsed_time    (month, year) float64 1kB dask.array<chunksize=(12, 15), meta=np.ndarray>
│         * injection_date  (injection_date) float64 32B 1.0 4.0 7.0 10.0
│         * month           (month) float64 96B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
│         * polygon_id      (polygon_id) float64 6kB 0.0 1.0 2.0 ... 687.0 688.0 689.0
│           time            (month, year) object 1kB dask.array<chunksize=(12, 15), meta=np.ndarray>
│           ...              ...
│         * y               (y) float32 512B 1.988e+07 1.957e+07 ... -1.988e+07
│         * year            (year) float64 120B 1.0 2.0 3.0 4.0 ... 12.0 13.0 14.0 15.0
│         * z_t_150m        (z_t_150m) float32 60B 500.0 1.5e+03 ... 1.35e+04 1.45e+04
│         * z_w             (z_w) float32 240B 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
│         * z_w_bot         (z_w_bot) float32 240B 1e+03 2e+03 ... 5.25e+05 5.5e+05
│         * z_w_top         (z_w_top) float32 240B 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
│       Dimensions without coordinates: y_b, x_b
│       Data variables:
│           ALK             (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
│           DIC             (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
│           FG              (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
│           Omega_arag      (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
│           Omega_calc      (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
│           PH              (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
│           lat             (y, x) float32 66kB dask.array<chunksize=(128, 128), meta=np.ndarray>
│           lat_b           (y_b, x_b) float32 67kB dask.array<chunksize=(129, 129), meta=np.ndarray>
│           lon             (y, x) float32 66kB dask.array<chunksize=(128, 128), meta=np.ndarray>
│           lon_b           (y_b, x_b) float32 67kB dask.array<chunksize=(129, 129), meta=np.ndarray>
│           pCO2SURF        (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
│       Attributes:
│           multiscales:  [{'datasets': [{'crs': 'EPSG:3857', 'level': 0, 'path': '.'...
└── Group: /1
        Dimensions:         (polygon_id: 690, band: 2, injection_date: 4, month: 12,
                             year: 15, y: 256, x: 256, y_b: 257, x_b: 257,
                             z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
        Coordinates: (12/13)
          * band            (band) <U12 96B 'delta' 'experimental'
            elapsed_time    (month, year) float64 1kB dask.array<chunksize=(12, 15), meta=np.ndarray>
          * injection_date  (injection_date) float64 32B 1.0 4.0 7.0 10.0
          * month           (month) float64 96B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
          * polygon_id      (polygon_id) float64 6kB 0.0 1.0 2.0 ... 687.0 688.0 689.0
            time          

In [41]:
dt["0"].ds

<xarray.DatasetView> Size: 456GB
Dimensions:         (polygon_id: 690, band: 2, injection_date: 4, month: 12,
                     year: 15, y: 128, x: 128, y_b: 129, x_b: 129,
                     z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates: (12/13)
  * band            (band) <U12 96B 'delta' 'experimental'
    elapsed_time    (month, year) float64 1kB dask.array<chunksize=(12, 15), meta=np.ndarray>
  * injection_date  (injection_date) float64 32B 1.0 4.0 7.0 10.0
  * month           (month) float64 96B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
  * polygon_id      (polygon_id) float64 6kB 0.0 1.0 2.0 ... 687.0 688.0 689.0
    time            (month, year) object 1kB dask.array<chunksize=(12, 15), meta=np.ndarray>
    ...              ...
  * y               (y) float32 512B 1.988e+07 1.957e+07 ... -1.988e+07
  * year            (year) float64 120B 1.0 2.0 3.0 4.0 ... 12.0 13.0 14.0 15.0
  * z_t_150m        (z_t_150m) float32 60B 500.0 1.5e+03 ... 1.35e+04 1.45e+04
  * z_w             (z_w) float32 240B 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot         (z_w_bot) float32 240B 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * z_w_top         (z_w_top) float32 240B 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
Dimensions without coordinates: y_b, x_b
Data variables:
    ALK             (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
    DIC             (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
    FG              (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
    Omega_arag      (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
    Omega_calc      (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
    PH              (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
    lat             (y, x) float32 66kB dask.array<chunksize=(128, 128), meta=np.ndarray>
    lat_b           (y_b, x_b) float32 67kB dask.array<chunksize=(129, 129), meta=np.ndarray>
    lon             (y, x) float32 66kB dask.array<chunksize=(128, 128), meta=np.ndarray>
    lon_b           (y_b, x_b) float32 67kB dask.array<chunksize=(129, 129), meta=np.ndarray>
    pCO2SURF        (polygon_id, band, injection_date, month, year, y, x) float32 65GB dask.array<chunksize=(1, 1, 1, 1, 15, 128, 128), meta=np.ndarray>
Attributes:
    multiscales:  [{'datasets': [{'crs': 'EPSG:3857', 'level': 0, 'path': '.'...

In [43]:
single_ds

<xarray.Dataset> Size: 625MB
Dimensions:         (polygon_id: 1, injection_date: 1, elapsed_time: 180,
                     nlat: 384, nlon: 320)
Coordinates:
    AREA_M2         (nlat, nlon) float64 983kB dask.array<chunksize=(192, 160), meta=np.ndarray>
    KMT             (nlat, nlon) float64 983kB dask.array<chunksize=(192, 160), meta=np.ndarray>
    TLAT            (nlat, nlon) float64 983kB dask.array<chunksize=(192, 160), meta=np.ndarray>
    TLONG           (nlat, nlon) float64 983kB dask.array<chunksize=(192, 160), meta=np.ndarray>
    time_delta      (elapsed_time) timedelta64[ns] 1kB dask.array<chunksize=(180,), meta=np.ndarray>
  * polygon_id      (polygon_id) int32 4B 0
  * injection_date  (injection_date) int32 4B 4
  * elapsed_time    (elapsed_time) int32 720B 0 1 2 3 4 ... 175 176 177 178 179
Dimensions without coordinates: nlat, nlon
Data variables: (12/13)
    ALK_ADD_SURF    (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    ALK_FLUX        (polygon_id, injection_date, elapsed_time) float64 1kB dask.array<chunksize=(1, 1, 180), meta=np.ndarray>
    ALK_FLUX_TOTAL  (polygon_id, injection_date, nlat, nlon) float64 983kB dask.array<chunksize=(1, 1, 192, 160), meta=np.ndarray>
    ALK_SURF        (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    DIC_ADD_SURF    (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    DIC_ADD_TOTAL   (polygon_id, injection_date, elapsed_time) float64 1kB dask.array<chunksize=(1, 1, 180), meta=np.ndarray>
    ...              ...
    DIC_FLUX_TOTAL  (polygon_id, injection_date, nlat, nlon) float64 983kB dask.array<chunksize=(1, 1, 192, 160), meta=np.ndarray>
    DIC_SURF        (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    FG_CO2_ADD      (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    PH_ADD_SURF     (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    pCO2_ADD_SURF   (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 88MB dask.array<chunksize=(1, 1, 45, 96, 80), meta=np.ndarray>
    DOR_efficiency  (polygon_id, injection_date, elapsed_time) float64 1kB dask.array<chunksize=(1, 1, 180), meta=np.ndarray>
Attributes:
    case:     smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-04-01_...